# CS 282
### Programming Assignment 1
#### Item 1

Jan Lendl R. Uy

2019-00312

#### Sytem Specifications
- macOS Sequoia 15.0.1
- Macbook Air M1 (ARM), 8-Core CPU and 8-Core GPU

#### Notes when Running the Notebook
- For items 1d and 1e, press any key to terminate OpenCV2 windows.
  - OpenCV2 windows tend to be buggy when attempted to be closed. They are best terminated through the following:
    - For macOS: Click 'Force Quit' in Activity Monitor
    - For Windows: Click 'End Task' in Task Manager

In [5]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import cv2
import math

### 1(a)

Select a negative floating-point number. Take its absolute value, round it, and then take its ceiling and floor. Print the results to screen.

In [7]:
# x = -5.44444
x = float(input("Enter a negative floating-point number: "))
print(f"Chosen floating-point number: {x}")

y = abs(x)
print(f"Absolute value of {x}: {y}")

z = round(y, 2)
print(f"Rounded-off value of {y}: {z}")

# Note: opencv2 has no built-in functions for getting the ceiling and floor
print(f"Ceiling of {z}: {math.ceil(z)}")
print(f"Floor of {z}: {math.floor(z)}")

Chosen floating-point number: -5.4444
Absolute value of -5.4444: 5.4444
Rounded-off value of 5.4444: 5.44
Ceiling of 5.44: 6
Floor of 5.44: 5


### 1(b)

Generate a 3 ×4 matrix whose elements are random integers. Print the matrix.

In [8]:
# Note: opencv2 arrays are a wrapper of numpy arrays
# Thus, the usage of numpy for matrix operations is inevitable
import numpy as np

In [9]:
# Create base matrix using numpy first (since cv2.Mat wraps numpy arrays)
base_mtx = np.zeros((3, 4), dtype=np.float32)
dst_mtx = cv2.Mat(base_mtx)

# Generate random numbers (floating-point)
# Range of possible random values are from 0 to 10
cv2.randu(dst_mtx, 0, 10)

# Convert to integers
X = cv2.convertScaleAbs(dst_mtx)

print(f"Random 3x4 matrix:\n{X}")

Random 3x4 matrix:
[[5 2 4 8]
 [4 2 8 8]
 [3 7 5 8]]


### 1(c)

Declare matrix variables A,B, and integer variable c within OpenCV. Assign them with the following values:

In [10]:
# Convert list to NumPy array and then to cv2.Mat
# Note: opencv2 requires an input array of type numpy.array 
# for initialization
A_list = [
    [6, 0, 2, 2, 6, 5, 9, 7, 1],
    [1, 6, 4, 5, 4, 9, 6, 9, 2],
    [3, 5, 9, 5, 3, 7, 8, 3, 3]
]

A = cv2.Mat(np.array(A_list, dtype=np.float32))

B_list = [
    [9, 1, 9, 7, 0, 5, 2, 0, 3],
    [9, 2, 3, 4, 3, 6, 0, 2, 9],
    [7, 2, 6, 0, 3, 8, 4, 2, 1]
]

B = cv2.Mat(np.array(B_list, dtype=np.float32))

c = 0.7
d = 0.55

### 1(c)(i)

$cA+ (1−c)B+ d$

In [12]:
answer = cv2.addWeighted(A, c, B, (1-c), d)
print(f"cA + (1-c)B + d =\n{answer}")

cA + (1-c)B + d =
[[7.45 0.85 4.65 4.05 4.75 5.55 7.45 5.45 2.15]
 [3.95 5.35 4.25 5.25 4.25 8.65 4.75 7.45 4.65]
 [4.75 4.65 8.65 4.05 3.55 7.85 7.35 3.25 2.95]]


### 1(c)(ii)

$A^{-1}$

Note: Get Moore-Penrose pseudoinverse via SVD since A is not square

In [13]:
s, u, vt = cv2.SVDecomp(A)
s_reciprocal = 1/s.flatten()
S_inv = np.diag(s_reciprocal)

A_inv = vt.T @ S_inv @ u.T
print(f"A inverse =\n{A_inv}")

A inverse =
[[ 0.05945878 -0.05401656  0.01753457]
 [-0.05320862  0.04879997  0.01134174]
 [-0.0269275  -0.04142783  0.08788505]
 [-0.02188468  0.0187535   0.01635406]
 [ 0.04004977 -0.00269519 -0.01477109]
 [-0.0162382   0.04623588 -0.00497328]
 [ 0.05000122 -0.04173273  0.03101051]
 [ 0.02012855  0.07637076 -0.07149199]
 [-0.00914656 -0.00456109  0.02115661]]


### 1(c)(iii)

$eig(B)$

#### Notes
- Rectangular matrices do **not** have eigenvalues, but their singular values can be obtained via SVD.
- $\mathrm{singular\_values}(B) = sqrt(eig(BB^T))$

In [14]:
_, eigenvalues, _ = cv2.eigen(B @ B.T)
singular_values_B = np.sqrt(eigenvalues)
print(f"Singular values of B =\n{singular_values_B}")

Singular values of B =
[[23.978624 ]
 [ 7.4619093]
 [ 6.5073414]]


Operation above can be verified by directly obtaining the singular values through SVD as follows.

In [15]:
s_B, u_B, vt_B = cv2.SVDecomp(B)
print(f"Singular values of B =\n{s_B}")

Singular values of B =
[[23.978622 ]
 [ 7.461909 ]
 [ 6.5073414]]


### 1(c)(iv)

Solve the matrix equation $Ax=b$

In [16]:
b_list = [
    [3, 5, 5],
    [7, 2, 8],
    [4, 1, 6]
]

b = cv2.Mat(np.array(b_list, dtype=np.float32))

# Use A_inv obtained from item 1(c)(ii)
x = A_inv @ b
print(f"x =\n{x}")

x =
[[-0.12960133  0.20679535 -0.02963119]
 [ 0.22734086 -0.15710144  0.19240707]
 [-0.01923707 -0.1296081   0.06125021]
 [ 0.13103671 -0.05556234  0.13872898]
 [ 0.04219861  0.18008736  0.09006076]
 [ 0.25504342  0.00630747  0.2588563 ]
 [-0.01808339  0.19755116  0.10220735]
 [ 0.309013    0.18189228  0.2826569 ]
 [ 0.02525914 -0.03369838  0.04471815]]


### 1(d)

Create a 2D matrix with three channels of type byte and size 100-by-100 and set all values to zero. Draw a red rectangle between with corners (30,10) and (60,40). Display this image.

In [ ]:
# Initialize zero matrix 
# Channels are the third dimension
# Use np.uint8 for byte-type
img = np.zeros((100, 100, 3), dtype=np.uint8)

# Draw red rectangle
# Red channel is the last channel in BGR format
# Set red channel to 255 in rectangle area
# -1 means fill entire shape
cv2.rectangle(img, (30, 10), (60, 40), (0, 0, 255), -1)

# Display image
print("Displaying red rectangle")
cv2.imshow("Red Rectangle", img)
cv2.waitKey(0)

Displaying red rectangle


2025-03-02 14:29:57.067 Python[56241:21724738] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-02 14:29:57.067 Python[56241:21724738] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


113

### 1(e)

Copy the red channel in (d) above into another matrix for display as grayscale image.

In [18]:
# Only get red channel and discard green and blue channels
red_channel = img[:, :, 2].copy()

# Display the red channel as grayscale
print("Displaying red rectangle in grayscale")
cv2.imshow("Red Channel as Grayscale", red_channel)
cv2.waitKey(0)

Displaying red rectangle in grayscale


113

In [19]:
# Clean-up
cv2.destroyAllWindows()